In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp
from IPython.display import clear_output

language = 'id'

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False                
    results = model.process(image)              
    image.flags.writeable = True                 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_styled_landmarks(image, results):
    if results.multi_hand_landmarks:
        for index, hand_landmarks in enumerate(results.multi_hand_landmarks):
            if results.multi_handedness[index].classification[0].index == 0:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                                            mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                            ) 
            else:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                            )
    
def extract_keypoints(results):
    if results.multi_hand_landmarks:
        return np.array([[res.x, res.y, res.z] for res in
                results.multi_hand_landmarks[0].landmark]).flatten(), True
    return np.zeros(21*3), False

DATA_PATH = os.path.join('SIBI Dataset')
actions = np.array([folder for folder in os.listdir(DATA_PATH)])

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(1, 63)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.load_weights('SIBI.h5')

def showText(image, sentence):
    cv2.rectangle(image, (0,440), (640, 480), (245, 117, 16), -1)
    cv2.putText(image, ' '.join(sentence), (3,470), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

In [2]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, hands)
        
        draw_styled_landmarks(image, results)
        
        keypoints, visible = extract_keypoints(results)
        
        if visible:
            keypoints = keypoints.reshape(1, keypoints.shape[0])

            pred = model.predict(np.expand_dims(keypoints, axis=0))
            res = np.argmax(pred[0])
            
            clear_output(wait=True)
            showText(image, actions[res])
            
        cv2.imshow('OpenCV Feed', image)
        cv2.waitKey(1)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 36ms/step
